# **Especialização em Ciência de Dados - INF/UFRGS e SERPRO**
### Disciplina CD004 - Metodologia de Aprendizado de Máquina Supervisionado
#### *Profa. Mariana Recamonde-Mendoza (mrmendoza@inf.ufrgs.br)*
<br> 

---
***Observação:*** *Este notebook é disponibilizado aos alunos como complemento às aulas síncronas e aos slides preparados pela professora. Desta forma, os principais conceitos são apresentados no material teórico fornecido. O objetivo deste notebook é reforçar os conceitos e demonstrar questões práticas no uso de diferentes algoritmos e estratégias de Aprendizado de Máquina.*


---


<br>

## **Aula 06** - **Tópico: Pré-processamento de dados. Seleção de atributos**

<br>

**Objetivo deste notebook**: Explorar estratégias para mitigar o problema de desbalanceamento de classes, complementando o pipeline da aula anterior que realizava pré-processamento dos dados através da imputação de valores e transformação de dados (codificação, discretização, normalização).
<br>

---





##**Predição de 'churn' de clientes de serviço de telecomunicação**

Os dados que utilizaremos neste notebook se referem a uma empresa de telecomunicações fictícia que forneceu serviços de telefone residencial e Internet para clientes na Califórnia. O conjunto de dados possui informações sobre os serviços para os quais cada cliente se inscreveu (telefone, várias linhas, internet, segurança online, backup online, proteção de dispositivos, suporte técnico e streaming de TV e filmes), informações da conta do cliente (há quanto tempo eles são clientes, contrato, forma de pagamento, cobrança sem papel, cobranças mensais e cobranças totais) e informações demográficas sobre os clientes (sexo, faixa etária, se eles têm parceiros e dependentes). Há, ainda, uma coluna chamada 'churn', que indica os clientes que desistiram do contrato do serviço no último mês. O objetivo da tarefa preditiva é identificar o churn (i.e., saída) de clientes a partir das informações coletadas. Os dados podem ser acessados neste [link](https://www.kaggle.com/datasets/blastchar/telco-customer-churn).




---



In [ ]:
## Carregando as bibliotecas básicas necessárias
# A primeira linha é incluída para gerar os gráficos logo abaixo dos comandos de plot
%matplotlib inline              
import pandas as pd             # para análise de dados 
import matplotlib.pyplot as plt # para visualização de informações
import seaborn as sns           # para visualização de informações
import numpy as np              # para operações com arrays multidimensionais

###Carregando e inspecionando os dados

Primeiramente, vamos carregar algumas bibliotecas importantes do Python e os dados a serem utilizados neste estudo. Os dados são disponibilizados através de um link, que também pode ser diretamente acessado pelos alunos.

In [ ]:
## Bibliotecas para treinamento/avaliação de modelos
from sklearn.model_selection import RepeatedKFold, train_test_split, cross_validate, cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

sns.set()

In [ ]:
## Carregando os dados
df = pd.read_csv("https://drive.google.com/uc?export=view&id=10VrzI8mA2wPvkNIDaLzv-IglPU-Febtf")#,na_values="NA")
df  

In [ ]:
## Características gerais do dataset
print("O conjunto de dados possui {} linhas e {} colunas".format(df.shape[0], df.shape[1]))

A coluna *'Churn'* contém a classificação de cada instância. Vamos avaliar a distribuição de classes do problema.

In [ ]:
## Distribuição do atributo alvo
plt.hist(df['Churn'])
plt.title("Distribuição do atributo alvo")
plt.show()

In [ ]:
df.info()

O atributo customerID é único para cada instância e no geral não possui valor preditivo. Iremos removê-lo da análise.

In [ ]:
customer_ids = df['customerID']
df = df.drop(['customerID'], axis=1)

In [ ]:
##remove as duplicatas
df=df.drop_duplicates(keep='last')

Como atributos categóricos e numéricos podem demandar pré-processamentos diferentes, vamos separar os respectivos "nomes" em dois vetores distintos, facilitando a manipulação dos dados posteriormente.

In [ ]:
## Separa os atributos em vetores, de acordo com o tipo de dado (categórico ou numérico)
cat_columns=list(df.drop(['Churn'], axis=1).select_dtypes(include=["object"]).columns)
print(cat_columns)

num_columns=list(df.select_dtypes(include=["int64", "float64"]).columns)
print(num_columns)


---


### Criando conjuntos de treino e teste para avaliação de modelos


Antes de iniciar o treinamento do modelo, lembre-se que é recomendado sempre reservar uma porção dos dados para teste, a qual somente será utilizada para avaliação do modelo final (após todo o processo de treinamento e otimização de hiperparâmetros).

Vamos fazer esta divisão, separando 20% para teste. Entretanto, primeiro precisamos dividir os dados entre atributos (X) e classe (y). Também iremos codificar as classes Yes/No para 1/0.



In [ ]:
## Separa o dataset em duas variáveis: os atributos/entradas (X) e a classe/saída (y)
X = df.drop(['Churn'], axis=1)
y = df['Churn'].values

Faremos o mapeamento das classes Yes/No para 1/0. Por padrão, as funções de avaliação assumem que a classe 1 é a positiva/de interesse.

In [ ]:
## substitui No' por 0, 'Yes' por 1
y = np.array([0 if y=='No' else 1 for y in y]) 

In [ ]:
## Faz a divisão entre treino (80%) e teste (20%).
## O conjunto de treino representa os dados que serão usados
## ao longo do desenvolvimento do modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,stratify=y,random_state=42) 

---


## Pré-Processamento dos Dados

Continuaremos usando o conceito de Pipelines para realizar o pré-processamento de atributos. Serão aplicados os seguintes passos, conforme notebooks anteriores:

*   Atributos **numéricos**: imputação de valores faltantes (com o método KNN) e normalização
*   Atributos **categóricos**: imputação de valores faltantes (com a moda), one-hot encoding, e normalização.

**Observação:** Salienta-se que como o objetivo deste notebook é observar os resultados dos métodos de redução de dimensionalidade, a aplicação destes métodos e o subsequente treinamento de modelos não será realizado com nested/k-fold cross-validation, mas sim com um simples holdout. Isto facilita a manipulação dos dados e a visualização dos seus resultados. Na prática, os métodos de redução de dimensionalidade devem ser inseridos no Pipeline que será executado a cada iteração do processo de treinamento e validação dos modelos.


In [ ]:
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

## pipeline específico para os atributos numéricos
num_pipeline = Pipeline([
                         ('imputer', KNNImputer(weights="uniform"))])

## pipeline específico para os atributos categóricos
cat_pipeline = Pipeline([
                         ('imputer', SimpleImputer(strategy='most_frequent')),
                         ('encoder', OneHotEncoder(drop='if_binary', sparse=False))])

## ColumnTransformer para aplicar cada pipeline ao respectivo tipo de atributo
data_pipeline = ColumnTransformer([
                                   ('numerical', num_pipeline, num_columns),
                                   ('categorical', cat_pipeline, cat_columns)])

## define pipeline que une as transformações definidas anteriormente e aplica a 
## normalização com método StandardScaler() em todos os atributos
prep_pipeline = Pipeline([
                 ('data_transform', data_pipeline),
                 ('data_normalize',StandardScaler())])

## ajusta o pipeline a partir dos dados de treino, e na sequência aplica em 
## treino e teste separadamente
prep_pipeline.fit(X_train)
X_train_prep = prep_pipeline.transform(X_train)
X_test_prep = prep_pipeline.transform(X_test)

In [ ]:
# ajusta nome das columas e mostra dataframe após pré-processamento
columns = np.append(num_columns,prep_pipeline[0].named_transformers_['categorical']['encoder'].get_feature_names_out(cat_columns))
df_train_prep = pd.DataFrame(X_train_prep, columns=columns)
df_test_prep = pd.DataFrame(X_test_prep, columns=columns)

Visualizando o formato do conjunto de dados de treinamento após aplicação do Pipeline de pré-processamento de dados.

In [ ]:
df_train_prep

In [ ]:
## Características gerais do dataset após pré-processamento
print("O conjunto de dados possui {} linhas e {} colunas referentes a atributos".format(df_train_prep.shape[0], df_train_prep.shape[1]))



---



## Redução de dimensionalidade com seleção de atributos por Filtro

O módulo [sklearn.feature_selection](https://scikit-learn.org/stable/modules/feature_selection.html) possui uma série de funções para selecionar atributos, removendo aqueles que parecem não ser relevantes para uma determinada tarefa preditiva. As seções a seguir avaliam as estratégias baseadas em filtro usando o método SelectKBest(). Este método aplica um critério para ordenação dos atributos de acordo com sua relevância ou poder discriminativo e então retém apenas um subconjunto dos mais relevantes, de acordo com valor 'k' informado na chamada ao método.

Para classificação, o sklearn disponibiliza dois critérios para estimar importância de atributos: f_classif (baseado no ANOVA) e mutual_inf_classif (baseado na análise de informação mútua)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.metrics import f1_score
from sklearn.svm import SVC

f_selected = list()
fs_perf = []
for ii in range(1, df_train_prep.shape[1]):
  fs = SelectKBest(score_func=f_classif, k=ii)
  X_train_fsc = fs.fit_transform(X_train_prep, y_train)
  X_test_fsc = fs.transform(X_test_prep)

  cols = fs.get_support(indices=True)
  f_selected.append((str(ii),columns[cols]))

  clf_fs = SVC(kernel='rbf', C=0.1)
  clf_fs = clf_fs.fit(X_train_fsc,y_train)
  clf_fs_pred = clf_fs.predict(X_test_fsc)

  fs_perf.append(round(f1_score(y_test, clf_fs_pred),3))

In [ ]:
plt.figure(figsize=(12, 6))
plt.xlabel("Number of features selected")
plt.ylabel("F1-score")
plt.plot(range(1,40), fs_perf, color='steelblue', linestyle='dashed', marker='o', markerfacecolor='darkblue', markersize=10)
plt.show()

In [ ]:
print(f_selected[0])

In [ ]:
print(f_selected[0][1])
print(f_selected[1][1])
print(f_selected[2][1])
print(f_selected[3][1])

Com Mutual Information

In [ ]:
from sklearn.feature_selection import mutual_info_classif

f_selected = list()
fs_perf = []
for ii in range(1, df_train_prep.shape[1]):
  fs = SelectKBest(score_func=mutual_info_classif, k=ii)
  X_train_fsc = fs.fit_transform(X_train_prep, y_train)
  X_test_fsc = fs.transform(X_test_prep)

  cols = fs.get_support(indices=True)
  f_selected.append((str(ii),columns[cols]))

  clf_fs = SVC(kernel='rbf', C=0.1)
  clf_fs = clf_fs.fit(X_train_fsc,y_train)
  clf_fs_pred = clf_fs.predict(X_test_fsc)

  fs_perf.append(round(f1_score(y_test, clf_fs_pred),3))

In [ ]:
plt.figure(figsize=(12, 6))
plt.xlabel("Number of features selected")
plt.ylabel("F1-score")
plt.plot(range(1,40), fs_perf, color='steelblue', linestyle='dashed', marker='o', markerfacecolor='darkblue', markersize=10)
plt.show()

In [ ]:
print(f_selected[0][1])
print(f_selected[1][1])
print(f_selected[2][1])
print(f_selected[3][1])



---



## Redução de dimensionalidade com seleção de atributos por métodos embedded

Alguns algoritmos de aprendizado de máquina são capazes de atribuir uma importância a cada atributo durante o processo de treinamento. No sklearn, estas informações estão normalmente nos atributos *coef_* ou *feature_importances_*.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

clf = RandomForestClassifier(n_estimators=50,random_state=42)
clf = clf.fit(X_train_prep, y_train)
importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_], axis=0)

Para florestas aleatórias, por ser um ensemble de árvores, podemos analisar a variação das importâncias atribuídas entre as árvores.

In [ ]:
forest_importances = pd.Series(clf.feature_importances_, index=columns)

fig, ax = plt.subplots(figsize=(12, 6))
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")

Aplicando o método SelectFromModel sobre o Floresta Aleatória. Por padrão SelectFromModel selecionará os atributos cuja importância é maior que a importância média de todos os atributos.

In [ ]:
model = SelectFromModel(clf, prefit=True)
X_preproc_fs = model.transform(X_train_prep)

print(X_preproc_fs.shape[1])

In [ ]:
features_idx = model.get_support()
features_name = columns[features_idx]
print(features_name)



---



## Redução de dimensionalidade com seleção de atributos por métodos wrapper

O sklearn disponibiliza diferentes métodos de seleção de atributos por wrapper, incluindo o RFE. Além disso, a biblioteca fornece o método RFECV, que automaticamente determina o melhor tamanho de subconjunto de atributos para selecionar. A célula abaixo exemplifica o uso do RFECV. Os resultados estão condicionados ao uso do SVC linear como método de avaliação de importância de atributos.

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

svc = SVC(kernel='linear', C=0.1) ## apenas o SVM linear possui 'coef_' para estimar importância de atributos

min_features_to_select = 1  # Número mínimo de atributos para considerar

## declara a estrutura do RFE-CV
rfecv = RFECV(
    estimator=svc,
    step=1,
    cv=StratifiedKFold(n_splits = 2),
    scoring="f1",
    min_features_to_select=min_features_to_select,
)
rfecv.fit(X_train_prep, y_train)

In [ ]:
print("Número 'ótimo' de atributos : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure(figsize=(12, 6))
plt.xlabel("Number of features selected")
plt.ylabel("F1-score")
plt.plot(range(min_features_to_select, len(rfecv.grid_scores_) + min_features_to_select),rfecv.grid_scores_,)
plt.show()

Sumarizando os atributos e o resultado da seleção de atributos (se foi selecionado ou não, e o respectivo ranking)

In [ ]:
for i in range(X_train_prep.shape[1]):
	print('Column: %d (%s), Selected %s, Rank: %.3f' % (i, columns[i], rfecv.support_[i], rfecv.ranking_[i]))